# Teksti klassifitseerimise ülesanne

Nagu mainitud, keskendume lihtsale teksti klassifitseerimise ülesandele, mis põhineb **AG_NEWS** andmestikul. Ülesandeks on klassifitseerida uudiste pealkirjad ühte neljast kategooriast: Maailm, Sport, Äri ja Teadus/Tehnoloogia.

## Andmestik

See andmestik on integreeritud [`torchtext`](https://github.com/pytorch/text) moodulisse, seega saame sellele hõlpsasti ligi.


In [1]:
import torch
import torchtext
import os
import collections
os.makedirs('./data',exist_ok=True)
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
classes = ['World', 'Sports', 'Business', 'Sci/Tech']

Siin sisaldavad `train_dataset` ja `test_dataset` kogumikke, mis tagastavad vastavalt paare, mis koosnevad sildist (klassi number) ja tekstist, näiteks:


In [2]:
list(train_dataset)[0]

(3,
 "Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\\band of ultra-cynics, are seeing green again.")

Niisiis, prindime välja meie andmekogumi esimesed 10 uut pealkirja:


In [5]:
for i,x in zip(range(5),train_dataset):
    print(f"**{classes[x[0]]}** -> {x[1]}")


**Sci/Tech** -> Wall St. Bears Claw Back Into the Black (Reuters) Reuters - Short-sellers, Wall Street's dwindling\band of ultra-cynics, are seeing green again.
**Sci/Tech** -> Carlyle Looks Toward Commercial Aerospace (Reuters) Reuters - Private investment firm Carlyle Group,\which has a reputation for making well-timed and occasionally\controversial plays in the defense industry, has quietly placed\its bets on another part of the market.
**Sci/Tech** -> Oil and Economy Cloud Stocks' Outlook (Reuters) Reuters - Soaring crude prices plus worries\about the economy and the outlook for earnings are expected to\hang over the stock market next week during the depth of the\summer doldrums.
**Sci/Tech** -> Iraq Halts Oil Exports from Main Southern Pipeline (Reuters) Reuters - Authorities have halted oil export\flows from the main pipeline in southern Iraq after\intelligence showed a rebel militia could strike\infrastructure, an oil official said on Saturday.
**Sci/Tech** -> Oil prices soar to

Kuna andmekogumid on iteraatorid, peame andmete mitmekordseks kasutamiseks need loendiks teisendama:


In [3]:
train_dataset, test_dataset = torchtext.datasets.AG_NEWS(root='./data')
train_dataset = list(train_dataset)
test_dataset = list(test_dataset)

## Tokeniseerimine

Nüüd peame teksti teisendama **numbriteks**, mida saab esitada tensoritena. Kui soovime sõnatasemel esindust, peame tegema kahte asja:
* kasutama **tokenisaatorit**, et jagada tekst **tokeniteks**
* koostama nende tokenite **sõnavara**.


In [4]:
tokenizer = torchtext.data.utils.get_tokenizer('basic_english')
tokenizer('He said: hello')

['he', 'said', 'hello']

In [5]:
counter = collections.Counter()
for (label, line) in train_dataset:
    counter.update(tokenizer(line))
vocab = torchtext.vocab.vocab(counter, min_freq=1)

Kasutades sõnavara, saame hõlpsasti kodeerida oma tokeniseeritud stringi numbrite kogumiks:


In [19]:
vocab_size = len(vocab)
print(f"Vocab size if {vocab_size}")

stoi = vocab.get_stoi() # dict to convert tokens to indices

def encode(x):
    return [stoi[s] for s in tokenizer(x)]

encode('I love to play with my words')

Vocab size if 95810


[599, 3279, 97, 1220, 329, 225, 7368]

## Sõnade kott tekstiesitus

Kuna sõnad kannavad tähendust, saame mõnikord teksti tähenduse välja selgitada, vaadates lihtsalt üksikuid sõnu, olenemata nende järjekorrast lauses. Näiteks uudiste klassifitseerimisel viitavad sõnad nagu *ilm*, *lumi* tõenäoliselt *ilmaennustusele*, samas kui sõnad nagu *aktsiad*, *dollar* viitavad *finantsuudistele*.

**Sõnade koti** (BoW) vektoriesitus on kõige sagedamini kasutatav traditsiooniline vektoriesitus. Iga sõna on seotud vektori indeksiga, vektori element sisaldab sõna esinemiste arvu antud dokumendis.

![Pilt, mis näitab, kuidas sõnade koti vektoriesitust mälus kujutatakse.](../../../../../translated_images/et/bag-of-words-example.606fc1738f1d7ba9.webp) 

> **Note**: BoW-d võib mõelda ka kui kõigi teksti üksiksõnade ühekuumkodeeritud vektorite summat.

Allpool on näide, kuidas luua sõnade koti esitus, kasutades Scikit Learn Python'i teeki:


In [7]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[1, 1, 0, 2, 0, 0, 0, 0, 0]], dtype=int64)

AG_NEWS andmekogumi vektorkujutise põhjal bag-of-words vektori arvutamiseks saame kasutada järgmist funktsiooni:


In [20]:
vocab_size = len(vocab)

def to_bow(text,bow_vocab_size=vocab_size):
    res = torch.zeros(bow_vocab_size,dtype=torch.float32)
    for i in encode(text):
        if i<bow_vocab_size:
            res[i] += 1
    return res

print(to_bow(train_dataset[0][1]))

tensor([2., 1., 2.,  ..., 0., 0., 0.])


> **Märkus:** Siin kasutame globaalset `vocab_size` muutujat, et määrata sõnavara vaikimisi suurus. Kuna sõnavara suurus on sageli üsna suur, saame piirata sõnavara suurust kõige sagedasemate sõnadega. Proovi vähendada `vocab_size` väärtust ja käivita allolev kood, et näha, kuidas see mõjutab täpsust. Võid oodata mõningast täpsuse langust, kuid mitte dramaatilist, parema jõudluse nimel.


## BoW klassifikaatori treenimine

Nüüd, kui oleme õppinud, kuidas luua tekstist Bag-of-Words esitus, treenime selle põhjal klassifikaatori. Kõigepealt peame oma treeningandmestiku ümber töötlema nii, et kõik positsioonivektori esitused muudetakse Bag-of-Words esituseks. Seda saab teha, kui edastada `bowify` funktsioon `collate_fn` parameetrina standardsele torch `DataLoader`-ile:


In [21]:
from torch.utils.data import DataLoader
import numpy as np 

# this collate function gets list of batch_size tuples, and needs to 
# return a pair of label-feature tensors for the whole minibatch
def bowify(b):
    return (
            torch.LongTensor([t[0]-1 for t in b]),
            torch.stack([to_bow(t[1]) for t in b])
    )

train_loader = DataLoader(train_dataset, batch_size=16, collate_fn=bowify, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=bowify, shuffle=True)

Nüüd defineerime lihtsa klassifitseeriva närvivõrgu, mis sisaldab ühte lineaarset kihti. Sisendvektori suurus on võrdne `vocab_size`-ga ja väljundi suurus vastab klasside arvule (4). Kuna lahendame klassifitseerimisülesannet, on lõplikuks aktivatsioonifunktsiooniks `LogSoftmax()`.


In [22]:
net = torch.nn.Sequential(torch.nn.Linear(vocab_size,4),torch.nn.LogSoftmax(dim=1))

Nüüd defineerime standardse PyTorchi treeningtsükli. Kuna meie andmekogum on üsna suur, treenime õpetamise eesmärgil ainult ühe epohhi jooksul ja mõnikord isegi vähem kui ühe epohhi jooksul (parameetri `epoch_size` määramine võimaldab meil treeningut piirata). Samuti raporteerime treeningu ajal kogunenud treeningtäpsust; raporteerimise sagedus määratakse parameetriga `report_freq`.


In [24]:
def train_epoch(net,dataloader,lr=0.01,optimizer=None,loss_fn = torch.nn.NLLLoss(),epoch_size=None, report_freq=200):
    optimizer = optimizer or torch.optim.Adam(net.parameters(),lr=lr)
    net.train()
    total_loss,acc,count,i = 0,0,0,0
    for labels,features in dataloader:
        optimizer.zero_grad()
        out = net(features)
        loss = loss_fn(out,labels) #cross_entropy(out,labels)
        loss.backward()
        optimizer.step()
        total_loss+=loss
        _,predicted = torch.max(out,1)
        acc+=(predicted==labels).sum()
        count+=len(labels)
        i+=1
        if i%report_freq==0:
            print(f"{count}: acc={acc.item()/count}")
        if epoch_size and count>epoch_size:
            break
    return total_loss.item()/count, acc.item()/count

In [25]:
train_epoch(net,train_loader,epoch_size=15000)

3200: acc=0.8028125
6400: acc=0.8371875
9600: acc=0.8534375
12800: acc=0.85765625


(0.026090790722161722, 0.8620069296375267)

## BiGrammid, TriGrammid ja N-Grammid

Üks sõnakoti lähenemise piirang on see, et mõned sõnad kuuluvad mitmesõnalistesse väljenditesse. Näiteks sõnal 'hot dog' on täiesti erinev tähendus võrreldes sõnadega 'hot' ja 'dog' teistes kontekstides. Kui me esindame sõnu 'hot' ja 'dog' alati samade vektoritega, võib see meie mudelit segadusse ajada.

Selle probleemi lahendamiseks kasutatakse sageli **N-grammi esindusi** dokumentide klassifitseerimise meetodites, kus iga sõna, kahe- või kolmesõnalise kombinatsiooni sagedus on kasulik tunnus klassifikaatorite treenimiseks. Näiteks bigrammi esinduses lisame sõnavarasse kõik sõnapaarid, lisaks algsetele sõnadele.

Allpool on näide, kuidas luua bigrammi sõnakoti esindust, kasutades Scikit Learn'i:


In [26]:
bigram_vectorizer = CountVectorizer(ngram_range=(1, 2), token_pattern=r'\b\w+\b', min_df=1)
corpus = [
        'I like hot dogs.',
        'The dog ran fast.',
        'Its hot outside.',
    ]
bigram_vectorizer.fit_transform(corpus)
print("Vocabulary:\n",bigram_vectorizer.vocabulary_)
bigram_vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()


Vocabulary:
 {'i': 7, 'like': 11, 'hot': 4, 'dogs': 2, 'i like': 8, 'like hot': 12, 'hot dogs': 5, 'the': 16, 'dog': 0, 'ran': 14, 'fast': 3, 'the dog': 17, 'dog ran': 1, 'ran fast': 15, 'its': 9, 'outside': 13, 'its hot': 10, 'hot outside': 6}


array([[1, 0, 1, 0, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]],
      dtype=int64)

N-gram lähenemise peamine puudus on see, et sõnavara suurus hakkab väga kiiresti kasvama. Praktikas peame N-grami esitusviisi kombineerima mõne dimensioonide vähendamise tehnikaga, näiteks *embeddings*, mida arutame järgmises osas.

Et kasutada N-grami esitusviisi meie **AG News** andmestikus, peame looma spetsiaalse ngrami sõnavara:


In [27]:
counter = collections.Counter()
for (label, line) in train_dataset:
    l = tokenizer(line)
    counter.update(torchtext.data.utils.ngrams_iterator(l,ngrams=2))
    
bi_vocab = torchtext.vocab.vocab(counter, min_freq=1)

print("Bigram vocabulary length = ",len(bi_vocab))

Bigram vocabulary length =  1308842


Me võiksime kasutada sama koodi nagu ülalpool klassifikaatori treenimiseks, kuid see oleks väga mälumahukas. Järgmises osas treenime bigrammide klassifikaatorit, kasutades sisendvektoreid.

> **Märkus:** Jätta saab ainult need ngrammid, mis esinevad tekstis rohkem kui määratud arv kordi. See tagab, et harvad bigrammid jäetakse välja ja vähendab oluliselt dimensioonide arvu. Selleks seadke `min_freq` parameeter kõrgemale väärtusele ja jälgige sõnavara pikkuse muutust.


## Term Frequency Inverse Document Frequency TF-IDF

BoW-esituses on sõnade esinemised võrdselt kaalutud, sõltumata sõnast endast. Siiski on selge, et sagedased sõnad, nagu *a*, *in* jne, on klassifitseerimise jaoks palju vähem olulised kui spetsialiseeritud terminid. Tegelikult on enamikus NLP ülesannetes mõned sõnad olulisemad kui teised.

**TF-IDF** tähistab **term frequency–inverse document frequency** (termini sagedus–dokumendi pöördsagedus). See on bag of words'i variatsioon, kus binaarse 0/1 väärtuse asemel, mis näitab sõna esinemist dokumendis, kasutatakse ujuvpunkti väärtust, mis on seotud sõna esinemissagedusega korpuses.

Formaalsemalt on sõna $i$ kaal $w_{ij}$ dokumendis $j$ defineeritud järgmiselt:
$$
w_{ij} = tf_{ij}\times\log({N\over df_i})
$$
kus
* $tf_{ij}$ on sõna $i$ esinemiste arv dokumendis $j$, st BoW väärtus, mida oleme varem näinud
* $N$ on dokumentide arv kogumis
* $df_i$ on dokumentide arv, mis sisaldavad sõna $i$ kogu kogumis

TF-IDF väärtus $w_{ij}$ suureneb proportsionaalselt sõna esinemiste arvuga dokumendis ja on tasakaalustatud korpuses olevate dokumentide arvuga, mis sisaldavad seda sõna. See aitab korrigeerida fakti, et mõned sõnad esinevad sagedamini kui teised. Näiteks, kui sõna esineb *iga* dokumendis kogumis, siis $df_i=N$ ja $w_{ij}=0$, ning need terminid jäetakse täielikult kõrvale.

TF-IDF vektoriseerimist saab hõlpsasti luua Scikit Learn'i abil:


In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit_transform(corpus)
vectorizer.transform(['My dog likes hot dogs on a hot day.']).toarray()

array([[0.43381609, 0.        , 0.43381609, 0.        , 0.65985664,
        0.43381609, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        ]])

## Kokkuvõte

Kuigi TF-IDF esitlused annavad erinevatele sõnadele sageduskaalu, ei suuda need esitada tähendust ega järjekorda. Nagu kuulus lingvist J. R. Firth ütles 1935. aastal: „Sõna täielik tähendus on alati kontekstuaalne ja ühtegi tähenduse uurimist väljaspool konteksti ei saa tõsiselt võtta.” Kursuse käigus õpime hiljem, kuidas tekstist kontekstuaalset teavet keelemudelite abil tabada.



---

**Lahtiütlus**:  
See dokument on tõlgitud AI tõlketeenuse [Co-op Translator](https://github.com/Azure/co-op-translator) abil. Kuigi püüame tagada täpsust, palume arvestada, et automaatsed tõlked võivad sisaldada vigu või ebatäpsusi. Algne dokument selle algses keeles tuleks pidada autoriteetseks allikaks. Olulise teabe puhul soovitame kasutada professionaalset inimtõlget. Me ei vastuta selle tõlke kasutamisest tulenevate arusaamatuste või valesti tõlgenduste eest.
